In [1]:
import numpy as np
import pandas as pd

In [2]:
# Read in labelled sample 
df = pd.read_csv("C:\\Users\\ajaco\\Desktop\\repos\\noreallyimfine\\ebird-project\\labelled_bird_sample.csv")

print(df.shape)

df.head()

(104732, 17)


,name,observ_count,country,state,county,latitude,longitude,observ_date,month,season,county_state,State,CountyName,RegionName,bird_rarity,region_rarity,seas_reg_rare
0,Magnolia Warbler,2,United States,Illinois,Cook,41.775629,-87.583273,1995-08-27,8,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common,Uncommon,Uncommon
1,Great Blue Heron,2,United States,Illinois,Cook,41.956746,-87.913027,1999-07-24,7,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common,Common,Common
2,Purple Martin,6,United States,Illinois,Cook,41.707907,-87.886848,1965-06-19,6,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common,Uncommon,Uncommon
3,Common Yellowthroat,3,United States,Illinois,Cook,41.707907,-87.886848,1965-06-19,6,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common,Common,Common
4,Eastern Towhee,3,United States,Illinois,Cook,41.707907,-87.886848,1965-06-19,6,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common,Common,Uncommon


In [3]:
# Multi-Class Classification 
# Check our baseline - pick the most common class for everything


df.seas_reg_rare.value_counts(normalize=True)

Uncommon    0.615371
Common      0.339782
Rare        0.044848
Name: seas_reg_rare, dtype: float64

**Baseline** 

Choosing 'Uncommon' would land 61.5% accuracy. 


In [9]:
# Baseline Confusion Matrix
from sklearn.metrics import confusion_matrix

ImportError: cannot import name 'plot_confusion_matrix' from 'sklearn.metrics' (C:\Users\ajaco\Anaconda3\lib\site-packages\sklearn\metrics\__init__.py)

In [8]:
import sklearn; sklearn.__version__

'0.21.3'

In [6]:
y_true = df['seas_reg_rare']
y_pred = np.array(["Uncommon"] * len(df))

confusion_matrix(y_true, y_pred)

array([[    0,     0, 35586],
       [    0,     0,  4697],
       [    0,     0, 64449]], dtype=int64)

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [11]:
train, test = train_test_split(df, stratify=df['seas_reg_rare'])

train.shape, test.shape

((78549, 17), (26183, 17))

In [12]:
train.columns

Index(['name', 'observ_count', 'country', 'state', 'county', 'latitude',
       'longitude', 'observ_date', 'month', 'season', 'county_state', 'State',
       'CountyName', 'RegionName', 'bird_rarity', 'region_rarity',
       'seas_reg_rare'],
      dtype='object')

In [37]:
cat_features = ['name', 'RegionName']
num_features = ['season']
target = 'seas_reg_rare'

X_train = train[cat_features]
y_train = train[target]
X_test = test[cat_features]
y_test = test[target]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((78549, 2), (78549,), (26183, 2), (26183,))

### Solution to unmatched birds:

(Some birds aren't able to be encoded properly because they don't appear in both train and test sets)

    - The 'white egret sp.' can be mapped to Great Egret
    - 

In [35]:
df.query("name == 'white egret sp.'").index

df.at[93757, 'name'] = 'Great Egret'

In [56]:
counts = df.name.value_counts()
counts[counts == 1].index

Index(['Corn Crake', 'Western x Clark's Grebe (hybrid)',
       'Sutton's Warbler (hybrid)', 'Kauai Elepaio',
       'Wild Turkey (Domestic type)',
       'African Collared-Dove (Domestic type or Ringed Turtle-Dove)',
       'Trindade Petrel', 'Golden-winged/Blue-winged Warbler',
       'Common/Thick-billed Murre', 'Terek Sandpiper',
       ...
       'Craveri's Murrelet', 'White-winged Parakeet', 'Gray Francolin',
       'Rosy-billed Pochard', 'Red-footed Booby', 'Cliff/Cave Swallow',
       'Connecticut/MacGillivray's/Mourning Warbler',
       'Gadwall x American Wigeon (hybrid)', 'Oahu Elepaio',
       'Eurasian Dotterel'],
      dtype='object', length=118)

In [58]:
one_time_birds = counts[counts == 1].index.tolist()
len(one_time_birds)

118

In [62]:
df['one-timer'] = df['name'].apply(lambda x: False if x in one_time_birds else True)
df['one-timer'].value_counts()

True     104614
False       118
Name: one-timer, dtype: int64

In [63]:
df.shape

(104732, 18)

In [65]:
df = df[df['one-timer'] == True]

In [55]:
# Exploring the issue below of not being able to encode the features

test_missing = set(X_test.name.unique()) - set(X_train.name.unique())
train_missing = set(X_train.name.unique()) - set(X_test.name.unique())
missing = test_missing + train_missing
len(missing)

TypeError: unsupported operand type(s) for +: 'set' and 'set'

In [39]:
df.query("name == 'small falcon sp.'")

,name,observ_count,country,state,county,latitude,longitude,observ_date,month,season,county_state,State,CountyName,RegionName,bird_rarity,region_rarity,seas_reg_rare
68724,small falcon sp.,1,United States,Connecticut,New Haven,41.24858,-72.90293,1999-10-04,10,Fall,New HavenConnecticut,Connecticut,New Haven,Mid-Atlantic North,Rare,Rare,Rare


In [40]:
df.query("name == 'Merlin'")

,name,observ_count,country,state,county,latitude,longitude,observ_date,month,season,county_state,State,CountyName,RegionName,bird_rarity,region_rarity,seas_reg_rare
3074,Merlin,1,United States,Oklahoma,Sequoyah,35.382752,-94.490061,1996-04-25,4,Spring,SequoyahOklahoma,Oklahoma,Sequoyah,Appohzarka,Common,Rare,Rare
4804,Merlin,2,United States,California,Orange,33.702653,-118.050735,1994-12-11,12,Winter,OrangeCalifornia,California,Orange,West Coast,Common,Uncommon,Uncommon
5979,Merlin,1,United States,Tennessee,Shelby,35.059580,-90.141020,1984-09-15,9,Fall,ShelbyTennessee,Tennessee,Shelby,Deep South,Common,Uncommon,Uncommon
6013,Merlin,5,United States,Tennessee,Shelby,35.059580,-90.141020,1992-09-18,9,Fall,ShelbyTennessee,Tennessee,Shelby,Deep South,Common,Uncommon,Uncommon
6698,Merlin,1,United States,New Hampshire,Rockingham,42.997506,-70.941712,1997-08-23,8,Summer,RockinghamNew Hampshire,New Hampshire,Rockingham,Northeast,Common,Uncommon,Uncommon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95851,Merlin,1,United States,Utah,Juab,39.865343,-113.393618,1990-04-09,4,Spring,JuabUtah,Utah,Juab,Mountain West,Common,Uncommon,Rare
95976,Merlin,1,United States,Minnesota,Carlton,46.532284,-92.550373,1998-07-23,7,Summer,CarltonMinnesota,Minnesota,Carlton,Great Lakes,Common,Uncommon,Uncommon
97588,Merlin,2,United States,Massachusetts,Dukes,41.343710,-70.814770,1997-10-06,10,Fall,DukesMassachusetts,Massachusetts,Dukes,Northeast,Common,Uncommon,Uncommon
98646,Merlin,1,United States,Arkansas,Chicot,33.392466,-91.383591,1992-12-22,12,Winter,ChicotArkansas,Arkansas,Chicot,Deep South,Common,Uncommon,Uncommon


In [24]:
df.name.nunique()

917

In [28]:
df[df['name'].str.contains("Egret")]

,name,observ_count,country,state,county,latitude,longitude,observ_date,month,season,county_state,State,CountyName,RegionName,bird_rarity,region_rarity,seas_reg_rare
25,Great Egret,2,United States,Illinois,Cook,41.850030,-87.650050,1975-05-10,5,Spring,CookIllinois,Illinois,Cook,Great Lakes,Common,Uncommon,Uncommon
185,Great Egret,5,United States,Illinois,Cook,41.956746,-87.913027,1996-08-02,8,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common,Uncommon,Uncommon
426,Great Egret,8,United States,Illinois,Cook,42.114000,-88.161000,1993-06-06,6,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common,Uncommon,Uncommon
438,Great Egret,1,United States,Illinois,Cook,42.084198,-88.179966,1998-08-11,8,Summer,CookIllinois,Illinois,Cook,Great Lakes,Common,Uncommon,Uncommon
642,Cattle Egret,4,United States,Illinois,Cook,41.794352,-87.578909,1979-10-24,10,Fall,CookIllinois,Illinois,Cook,Great Lakes,Common,Uncommon,Rare
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104440,Great Egret,1,United States,Ohio,Highland,39.184901,-83.483391,1994-11-27,11,Fall,HighlandOhio,Ohio,Highland,Appohzarka,Common,Uncommon,Uncommon
104499,Great Egret,1,United States,Oklahoma,Johnston,34.185607,-96.650943,1991-07-20,7,Summer,JohnstonOklahoma,Oklahoma,Johnston,Appohzarka,Common,Uncommon,Common
104578,Cattle Egret,1,United States,Louisiana,Morehouse,32.731059,-91.996405,1982-11-22,11,Fall,MorehouseLouisiana,Louisiana,Morehouse,Deep South,Common,Uncommon,Uncommon
104634,Great Egret,10,United States,Ohio,Marion,40.568703,-83.274136,1998-09-26,9,Fall,MarionOhio,Ohio,Marion,Heartland,Common,Uncommon,Uncommon


In [25]:
# Problem - some birds appear only once, so can't be shared in both train and test. 

# I feel like we've dealt with this in class before, can't remember right now.

In [17]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()

X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)

ValueError: Found unknown categories ['Red-winged/Tricolored Blackbird', 'kinglet sp.', 'Spotted/Eastern Towhee (Rufous-sided Towhee)', 'White-faced Storm-Petrel', "Bulwer's Petrel", "Xantus's Hummingbird", 'Golden-winged/Blue-winged Warbler', 'Mangrove Cuckoo', 'longspur sp.', 'Common Sandpiper', 'ptarmigan sp.', 'diurnal raptor sp.', 'African Collared-Dove (Domestic type or Ringed Turtle-Dove)', 'dabbling duck sp.', 'Rosy-faced Lovebird', "Cory's/Great Shearwater", 'Carolina x Black-capped Chickadee (hybrid)', "Leach's/Townsend's Storm-Petrel (dark-rumped)", 'Yellow-billed/Black-billed Cuckoo', 'Gray Wagtail', 'Green Parakeet', 'Dusky Warbler', 'Hoary Redpoll', 'small falcon sp.', 'African Silverbill', 'Little Egret', 'white egret sp.', 'Rose-breasted/Black-headed Grosbeak', 'Red-footed Booby', "Common/Forster's Tern", 'Red-crowned Parrot', 'Rosy-billed Pochard', "Common/Barrow's Goldeneye"] in column 0 during transform

In [14]:
model = LogisticRegression()

model.fit(X_train, y_train)

preds = model.predict(X_test)

print("Accuracy of OOB Logistic Regression:", model.score(X_test))

C:\Users\ajaco\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: could not convert string to float: 'Mountain West'